In [247]:
import pandas as pd
import dateutil.parser
import re
import numpy as np

In [248]:
df = pd.read_csv('../Created CSVs/all_seins_all_data_standard_dates.csv')

In [249]:
df['Date'] =  [dateutil.parser.parse(date) for date in list(df['Date'])]

In [250]:
def fix_ph(pH_li):
    pH_list  = []
    pH_li = pH_li.fillna('')
    pH_lists  = [re.findall('\d+\.*', x ) for x in pH_li]

    for pH in pH_lists:
        if len(pH) == 0:
            pH_list.append(np.NaN)
        elif len(pH) ==  1:
            pH_list.append(pH[0])
            #print pH[0]
        elif pH[0][-1] == '.':
            pH_list.append(pH[0]+pH[1])
            #print pH[0]+pH[1]
        else:
            pH_list.append(pH[0])
            #print 'bad',pH[1], pH
    #pHCompare['pH'] = pH_list
    return [float(x) for x in pH_list]

In [251]:
df['Corrected pH'] = fix_ph(df['pH'])

In [252]:
def fix_water_temp(h20_temp_list):
    water_temp_list = []
    h20_temp_list = h20_temp_list.fillna('')
    h20_temp_list  = [re.findall('\d+\.*', x ) for x in h20_temp_list]
    for temp in h20_temp_list:
        #print temp
        if len(temp) == 0:
            water_temp_list.append(np.NaN)
        elif len(temp) ==  1:
            if int(temp[0]) < 30:
                new_temp = int(temp[0]) * 9.0/5 + 32
                water_temp_list.append(str(new_temp))
            else:
                water_temp_list.append(temp[0])
            #print pH[0]
        elif temp[0][-1] == '.':
            water_temp_list.append(temp[0]+temp[1])
            #print pH[0]+pH[1]
        else:
            water_temp_list.append(temp[0])
    
    return [float(x) for x in water_temp_list]

In [253]:
df["Corrected Water Temp:"] = fix_water_temp(df['Water Temp:'])

In [254]:
def fix_salinity(sal_list):
    salinity = []
    for sal in sal_list:
        if sal != sal:
            salinity.append(sal)
        elif any(char.isdigit() for char in sal) == False:
            salinity.append(np.NAN)
        else:
            first_num  = re.search(r'\d+', sal).group()
            salinity.append(first_num)
    return [float(x) for x in salinity]

In [255]:
df['Corrected Salinity'] = fix_salinity(df['Salinity'])

In [256]:
def fix_DO(DO_list):
    cleaned_list = []
    DO_list = DO_list.fillna('')
    DO_list  = [re.findall('\d+\.*', x ) for x in DO_list]

    for DO in DO_list:
        if len(DO) == 0:
            cleaned_list.append(np.NaN)
        else:
            cleaned_list.append(int(DO[0]))
       
    return cleaned_list

In [257]:
df['Corrected DO'] = fix_DO(df['DO'])

In [258]:
df['Corrected Quantity'] = df['Quantity'].fillna(1)

In [259]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [260]:
def fix_quantity(quant_list):
    num_caught_list = []
    for num in list(quant_list):
        if type(num) ==  int:
            num_caught_list.append(num)
        elif is_number(num):
            num_caught_list.append(int(float(num)))
        else:
            number = ''
            started = False
            for char in num:
                if char.isdigit():
                    number += char
                    started = True
                else:
                    if char == '~':
                        print started
                    if started:
                        break
            if len(number):
                num_caught_list.append(int(number))
            else:
                if num in ['Numerous', 'TMTC', 'Too Many To Count']:
                    num_caught_list.append(30)
                else:
                    num_caught_list.append(1)
    return num_caught_list

In [261]:
df['Corrected Quantity'] = fix_quantity(df['Corrected Quantity'])

False
False
False


In [262]:
def fix_hauls(hauls_li):
    #?, Many, and Unknown have all been gotten rid of and not replaced by a number.
    clean_hauls = []
    for haul in hauls_li:
        if haul != haul:
            clean_hauls.append(np.nan)
        elif any(char.isdigit() for char in haul):
            first_num  = re.search(r'\d+', haul).group()
            clean_hauls.append(first_num)
        else:
            clean_hauls.append(np.nan)
    return clean_hauls

In [263]:
df['Corrected # Hauls'] = fix_hauls(df['# Hauls'])

In [264]:
remove_row_list = ['two hemigrapsis molts were found on the beach', 'species caught', 'slipper snail shell', 'slipper snail shell', 'slipper shell', 'oyster shell','hemigraphis molt']

In [265]:
fish_list = [re.sub(r'([^\s\w]|_)+', '', f) for f in df['Common Name']]
fish_list = [f.lower() for f in fish_list]
fish_list = [f.strip() for f in fish_list]

In [266]:
df['Common Name'] = fish_list

In [267]:
df = df[df['Common Name'].isin(remove_row_list) == False]

In [268]:
df = df.reset_index()

In [269]:
fish_list = [f.replace('yoy', '') for f in df['Common Name']]

In [270]:
from difflib import SequenceMatcher

In [271]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [272]:
count = 0
fish_changes = []
for i in range(len(fish_list)):
    for j in range(i+1, len(fish_list)):
        if fish_list[i] ==  fish_list[j]:
            continue
        if similar(fish_list[i], fish_list[j]) > 0.85:
            fish_changes.append([fish_list[i], fish_list[j]])
            if len(fish_list[i]) < len(fish_list[j]):
                fish_list[j] = fish_list[i]
            else:
                fish_list[i] = fish_list[j]

In [284]:
def make_name_corrected_names(fish_list):
    name_changes = {'1 oyster toadfish caught in traffic cone' : 'oyster toadfish', '20 ctenophores': 'ctenophore',
               'atlantic silverside menidia menidia': 'atlantic silverside','atlanticsilverside': 'atlantic silverside',
               'blackfish tautog': 'tautog', 'blue muscle': 'blue mussel', 'bluefish juvenile': 'bluefish',
               'comb jellies': 'comb jelly','comb jellies ctenophores': 'comb jelly','common shore shrimp': 'shore shrimp',
               'common spider crab': 'spider crab', 'bluecrab': 'blue crab', 'crab blue crab':'blue crab', ' crab blue crab': 'blue crab',
               'juvenille stripped bass': 'striped bass', 'jvvenilee striped buss': 'striped bass', 'taotog': 'tautog',
               'tautogblackfish': 'tautog', 'the cutest northern puffer ever':'northern puffer','window pain':'windowpane flounder',
               'true jellies': 'jelly fish', 'true jelly': 'jelly fish', 'true jelly fish': 'jelly fish', 'sea squirtsolitary tunicate': 'sea squirt',
               'oyster drill': 'oyster drill snail', 'green crab carcinus maenas': 'green crab','green crab european':'green crab',
               'porgyscup': 'porgy', 'norther puffer': 'northern puffer', 'norther pipefish': 'northern pipefish', 'ctenophore':'comb jelly',
                'ghost shrimp':'glass shrimp','glass shrimp shore shrimp': 'glass shrimp', 'golden star colonial tunicate': 'golden star tunicate',
               'lions mane jellyfish cyanea capillata': 'lions mane jellyfish', 'muslces':'mussel', 'pipefish syngnathinae': 'northern pipefish',
               "porgies":"porgy", "sea horse hippocampus": "seahorse", "sea horse": "seahorse", 'winter flounder righteye': 'winter flounder',
               'shrimp seven spine': 'seven spine shrimp', 'star tunicate':'golden star tunicate', 'seven spine shrimp': 'sand shrimp',
               'puffer': 'northern puffer', 'northern sea robin': 'sea robin', 'herring spp pre metamorph': 'herring', 'grubby sculpin family': 'grubby',
               "blue back": "blueback herring", "anchovy": "bay anchovy", "decoration crab": "spider crab", "flatfish": "unidentified flatfish","flounder":"unidentified flatfish",
               "flounder righteye": "winter flounder", "glass shrimp": "shore shrimp", "grass shrimp": "sand shrimp","grubby sculpin family": "grubby",
               "hermit crab": "unidentified hermit crab", "herring": "unidentified herring", "herring spp pre metamorph":"unidentified herring",
               "hydra medusa": "hydrozoa", "hydroid": "hydrozoa", "hydromedusae": "hydrozoa", "jelly fish": "unidentified jelly fish", "menhaden": "atlantic menhaden",
               "mussel": "unidentified mussel", "oyster":"eastern oyster", "pipefish":"northern pipefish", "poly worm": "sand worm", "sea bass": "black sea bass",
               "sea robin": "northern sea robin", "sea snail": "unidentified sea snail", "seahorse": "northern seahorse", "shrimp": "unidentified shrimp",
               "shrimp shore and mud": "unidentified shrimp", "silverside": "atlantic silverside", "snail": "unidentified snail","snail eggs": "unidentified snail eggs",
               "sponge": "unidentified sponge", "tunicates": "tunicate", "tunicate sponge": "tunicate",' american shad': 'american shad'}
    for i in range(len(fish_list)):
        if fish_list[i] in name_changes:
            fish_list[i] = name_changes[fish_list[i]]
    return fish_list

In [285]:
def make_name_corrected_and_grouped_names(fish_list):
    name_changes = {'1 oyster toadfish caught in traffic cone' : 'oyster toadfish', '20 ctenophores': 'comb jelly',
               'atlantic silverside menidia menidia': 'atlantic silverside','atlanticsilverside': 'atlantic silverside',
               'blackfish tautog': 'tautog', 'blue muscle': 'mussel', 'bluefish juvenile': 'bluefish',
               'comb jellies': 'comb jelly','comb jellies ctenophores': 'comb jelly','common shore shrimp': 'shrimp',
               'common spider crab': 'spider crab', 'bluecrab': 'blue crab', 'crab blue crab':'blue crab', ' crab blue crab': 'blue crab', 
               'juvenille stripped bass': 'striped bass', 'jvvenilee striped buss': 'striped bass', 'taotog': 'tautog',
               'tautogblackfish': 'tautog', 'the cutest northern puffer ever':'puffer','window pain':'flatfish',
               'true jellies': 'jellyfish', 'true jelly': 'jellyfish', 'true jelly fish': 'jellyfish', 'sea squirtsolitary tunicate': 'sea squirt',
               'oyster drill': 'sea snail', 'green crab carcinus maenas': 'green crab','green crab european':'green crab',
               'porgyscup': 'porgy', 'norther puffer': 'puffer', 'norther pipefish': 'northern pipefish', 'ctenophore':'comb jelly',
                'ghost shrimp':'shrimp','glass shrimp shore shrimp': 'shrimp', 'golden star colonial tunicate': 'tunicates',
               'lions mane jellyfish cyanea capillata': 'jellyfish', 'muslces':'mussel', 'pipefish syngnathinae': 'northern pipefish',
               "porgies":"porgy","seahorse":"northern seahorse", "sea horse hippocampus": "northern seahorse", "sea horse": "northern seahorse", 'winter flounder righteye': 'flatfish',
               'shrimp seven spine': 'shrimp', 'star tunicate':'tunicates', 'seven spine shrimp': 'shrimp','pipefish': 'northern pipefish',
               'northern puffer': 'puffer', 'northern sea robin': 'sea robin', 'herring spp pre metamorph': 'herring', 'grubby sculpin family': 'grubby',
               "blue back": "herring", 'alewife':'herring', 'atlantic herring': 'herring', 'blueback herring': 'herring',
               'colonial tunicate': 'tunicates', 'golden star tunicate': 'tunicates','sea squirt': 'tunicates', 'tunicate sponge': 'tunicates',
               'shrimp shore and mud': 'shrimp', 'seven spine shrimp': 'shrimp', 'shore shrimp': 'shrimp', 'sand shrimp': 'shrimp', 'mantis shrimp': 'shrimp',
               'grass shrimp': 'shrimp', 'glass shrimp': 'shrimp', 'flounder': 'flatfish', 'flounder righteye': 'flatfish', 'hogchoker': 'flatfish',
               'summer flounder': 'flatfish', 'windowpane flounder': 'flatfish', 'winter flounder': 'flatfish', 'menhaden': 'atlantic menhaden',
               'silverside': 'atlantic silverside', 'blue mussel': 'mussel', 'eastern oyster': 'oyster', 'anchovy': 'bay anchovy',
                'moon jelly': 'jellyfish', 'jelly fish': 'jellyfish', 'lions mane jellyfish': 'jellyfish', 'scud': 'amphipod', 'ribbed mussel': 'mussel',
               'mud snail': 'sea snail', 'oyster drill snail': 'sea snail', 'snail': 'sea snail', 'whelk snail':'sea snail', 'snail eggs': 'sea snail', 'sea bass': 'black sea bass',
               'sheepshead porgy': 'porgy', 'striped searobin': 'sea robin', 'black fingered mud crab': 'mud crab', 'clam worm probably nereis spp': 'polychaete worm',
               'poly worm': 'polychaete worm', 'sandworm': 'polychaete worm', 'white fingered mud crab': 'mud crab', 'decoration crab': 'spider crab',
               'hydra medusa': 'hydrozoa', 'hydroid': 'hydrozoa', 'hydromedusae' : 'hydrozoa', 'red beard sponge': 'sponge', ' american shad': 'american shad'}
    for i in range(len(fish_list)):
        if fish_list[i] in name_changes:
            fish_list[i] = name_changes[fish_list[i]]
    return fish_list

In [287]:
nogroup = make_name_corrected_names(fish_list[:])

In [288]:
group = make_name_corrected_and_grouped_names(fish_list[:])

In [289]:
len(group)

892

In [292]:
len([i for i, j in zip(nogroup, list(df3['Species'])) if i == j])

892

In [291]:
for i in range(len(df2)):
    if nogroup[i] != list(df3['Species'])[i]:
        print "error no group", '+'+nogroup[i],'+', list(df3['Species'])[i],'+', fish_list[i]

    if group[i] != list(df2['Species'])[i]:
        print "error group", group[i], list(df2['Species'])[i], fish_list[i]

        

In [195]:
#df2 = pd.read_csv('../Created CSVs/Combined_Name_Corrected_All_Seins.csv')

In [293]:
df['Grouped Corrected Common Name'] = group#df2['Species']

In [294]:
#df3 = pd.read_csv('../Created CSVs/Non_Combined_Name_Corrected_All_Seins.csv')

In [295]:
df['Corrected Common Name'] = nogroup#df3['Species']

#### not yet cleaned: Time, Weather & Air Temp, Tide Times, Tide is Moving, Turbidity, # student seiners, # visitors, instructors, Scientific Name, Brought to Ed Center, Other Info, Notes from Seining
#### Of these Weather & Air Temp, Tide Times Tide is Moving and Turbidity should be done. The rest probably don't matter
#### I'd also like to see if we can pull from outside data for Weather & Air Temp, Water Temp, Tide Times, Tide is Moving, DO, pH, Salinity, and Turbidity. If so we could get better readings.

#### Time should be start time, end time, length of seine - 3 columns, Weather and Air Temp should be 2 Weather and Air Temp, plus both from other origin.

#### if you wanted to make a  further interesting and easy to work with csv, you can make one that combines all the animals caught list, and amount caught into one row with a comma separated string.

In [296]:
df.to_csv('../Created CSVs/Cleaned_For_Analysis_All_Data.csv')

In [297]:
#pd.options.display.max_rows = 999

In [298]:
#Looks good!
#df[['Salinity', 'Corrected Salinity', 'DO', 'Corrected DO', 'pH', 'Corrected pH', 'Quantity', 'Corrected Quantity', '# Hauls', 'Corrected # Hauls', 'Water Temp:', 'Corrected Water Temp:']]

In [302]:
#checks out!
#df[['Date','Corrected Quantity','Common Name', 'Corrected Common Name', 'Grouped Corrected Common Name']]

,index,Unnamed: 0,Date,Time,Weather & Air Temp:,Water Temp:,Tide Times,Tide is moving,DO,pH,...,"Other info (Gender, Length, Dead/Alive, other physical description)",NOTES FROM SEINEING SEASON:,Corrected pH,Corrected Water Temp:,Corrected Salinity,Corrected DO,Corrected Quantity,Corrected # Hauls,Grouped Corrected Common Name,Corrected Common Name
0,0,0,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,tunicates,sea squirt
1,1,1,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,shrimp,shore shrimp
2,2,2,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,shrimp,mantis shrimp
3,3,3,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,shrimp,sand shrimp
4,4,4,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,hermit crab,unidentified hermit crab
5,5,5,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,oyster toadfish,oyster toadfish
6,6,6,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,striped bass,striped bass
7,7,7,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,northern kingfish,northern kingfish
8,8,8,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,northern pipefish,northern pipefish
9,9,9,2008-09-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,sea robin,striped searobin
